In [1]:
import math
import csv
import json

STAT_NAMES = [
    "caps", "garbage_time_caps", "hold", "ndps", "returns", "quick_returns", "nrts", "pups", 
    "keypops", "handoffs", "goodprevent", "resets", "badflaccids", "sparkedouts"
]

with open("smurfs.json") as f:
    smurfs = json.load(f)

def desmurf(player):
    player = player.replace("\\'", "'").replace("\\\\", "\\")
    if player in smurfs:
        return smurfs[player]
    return player

def row_to_dict(row):
    return {
        'match_id': int(row[0]),
        'timestamp': int(row[1]),
        'duration': int(row[2]),
        'cap_diff': int(row[3]),
        'garbage_time_cap_diff': int(row[4]),
        'players': [
            {
                'name': desmurf(row[j + 5]),
                'stats': {
                    stat_name: int(row[j * len(STAT_NAMES) + 13 + i])
                    for i, stat_name in enumerate(STAT_NAMES)
                }
            }
            for j in range(8)
        ]
    }

with open("matchups_with_stats.csv") as f:
    reader = csv.reader(f)
    next(reader)
    matches = [row_to_dict(row) for row in reader]

elo = {}
variance = {}
pred_errors = []
winner_win_prob = []

In [2]:
STARTING_VARIANCE = 1.2
VARIANCE_TO_ADD_BACK = 0.002
BASE_VARIANCE = 4.5
RED_ADVANTAGE = 0.1
PLAYER_STAT_WEIGHTS_INITIAL = {
    "caps": 0.8,
    "garbage_time_caps": -0.3,
    "hold": 1.7 / 3600,
    "ndps": -0.5,
    "returns": 0.35,
    "quick_returns": 0.0,
    "nrts": 0.3,
    "pups": 0.3,
    "keypops": 0.0,
    "handoffs": 0.2,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_INITIAL = {
    "caps": 0,
    "garbage_time_caps": -0.3,
    "hold": 0.8 / 3600,
    "ndps": -0.0,
    "returns": 0.04,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
PLAYER_STAT_WEIGHTS_CONVERGED = {
    "caps": 0.05,
    "garbage_time_caps": -0.0,
    "hold": 0.0 / 3600,
    "ndps": -0.0,
    "returns": 0.0,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_CONVERGED = {
    "caps": 0,
    "garbage_time_caps": -0.0,
    "hold": 0.4 / 3600,
    "ndps": -0.0,
    "returns": 0.02,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": 0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": 0.0,
    "sparkedouts": 0.0
}
DIFF_MAPPING = [0, 0.2, 0.9, 2.1, 3.1, 4.2, -4.2, -3.1, -2.1, -0.9, -0.2]
NEW_PLAYER_ELO = -1.1
NEWNESS_THRESHOLD = 0.05
NEW_PLAYER_ELO_BOOST = 0.1
DIFF_WEIGHT_INITIAL = 0.65
DIFF_WEIGHT_CONVERGED = 0.6
GARBAGE_TIME_DISCOUNT = 0.35
RELATIVE_ELO_CORRECTION = 0.05

In [3]:
def get_elo(player):
    if player not in elo:
        elo[player] = NEW_PLAYER_ELO
    return elo[player]

def get_variance(player):
    if player not in variance:
        variance[player] = STARTING_VARIANCE
    return variance[player]

def normalize_elos():
    known_elos = [elo[p] for p in elo if variance[p] <= 0.2]
    avg_elo = sum(known_elos) / max(10, len(known_elos))
    for p in elo:
        elo[p] -= avg_elo
        variance[p] = min(STARTING_VARIANCE, variance[p] + VARIANCE_TO_ADD_BACK)

In [4]:
def judge_stats_initial(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def judge_stats_converged(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def update_rating(player, error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg):
    share_of_variance = variance[player] / total_variance
    newness = variance[player] / STARTING_VARIANCE
    error_weight = DIFF_WEIGHT_INITIAL * newness + DIFF_WEIGHT_CONVERGED * (1 - newness)
    stat_score = stat_score_initial * newness + stat_score_converged * (1 - newness)
    update = error_weight * error + stat_score
    update -= elo_vs_game_avg * RELATIVE_ELO_CORRECTION
    
    elo[player] += update * share_of_variance
    variance[player] *= 1 - share_of_variance
    if variance[player] > NEWNESS_THRESHOLD:
        elo[player] += NEW_PLAYER_ELO_BOOST * share_of_variance

def update_ratings(match):
    expected_score = RED_ADVANTAGE
    total_variance = BASE_VARIANCE
    
    for player in match['players'][:4]:
        expected_score += get_elo(player['name'])
        total_variance += get_variance(player['name'])
    for player in match['players'][4:]:
        expected_score -= get_elo(player['name'])
        total_variance += get_variance(player['name'])
    
    all_player_names = [p['name'] for p in match['players']]
    all_player_elos = [get_elo(name) for name in all_player_names]
    game_avg_elo = sum(all_player_elos) / len(all_player_elos)

    winner_win_prob.append(1 / (1 + math.exp(-expected_score / 2 if match['cap_diff'] > 0 else expected_score / 2)))
    pred_errors.append(match['cap_diff'] - min(max(expected_score, -5), 5))
    error = DIFF_MAPPING[match['cap_diff']] -\
        match['garbage_time_cap_diff'] * GARBAGE_TIME_DISCOUNT -\
        min(max(expected_score, DIFF_MAPPING[-5]), DIFF_MAPPING[5])
    
    red_stats_initial, blue_stats_initial = judge_stats_initial(match['players'])
    red_stats_converged, blue_stats_converged = judge_stats_converged(match['players'])
    for player, stat_score_initial, stat_score_converged in zip(match['players'][:4], red_stats_initial, red_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)
    for player, stat_score_initial, stat_score_converged in zip(match['players'][4:], blue_stats_initial, blue_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], -error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)

In [5]:
elo = {}
variance = {}
pred_errors = []
winner_win_prob = []
day = 0

for match in matches:
    new_day = round(match['timestamp'] / 86400)
    if new_day != day:
        normalize_elos()
    day = new_day
    update_ratings(match)

print(f"MAE: {sum([abs(e) for e in pred_errors]) / len(pred_errors):.4f}    MSE: {sum([e ** 2 for e in pred_errors]) / len(pred_errors):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob]) / len(winner_win_prob):.2%}    WWP: {sum(winner_win_prob) / len(winner_win_prob):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob]) / len(winner_win_prob)) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob]) / len(winner_win_prob)):.2%}")

MAE: 2.3876    MSE: 8.3115
COR: 59.58%    WWP: 52.63%
BRI: 51.48%    LOG: 51.54%


In [6]:
print(f"MAE: {sum([abs(e) for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}    MSE: {sum([e ** 2 for e in pred_errors[10000:]]) / len(pred_errors[10000:]):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:]):.2%}    WWP: {sum(winner_win_prob[10000:]) / len(winner_win_prob[10000:]):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob[10000:]]) / len(winner_win_prob[10000:])):.2%}")

MAE: 2.3846    MSE: 8.2845
COR: 58.70%    WWP: 52.17%
BRI: 51.20%    LOG: 51.23%


In [7]:
leaderboard = sorted([
    p for p in elo if variance[p] <= 0.3
], key=lambda x: elo[x], reverse=True)
for i, player in enumerate(leaderboard):
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}")

  1.  okthen         2.73 ± 0.67
  2.  SluffAndRuff   2.71 ± 0.62
  3.  CarrotCake     2.70 ± 0.30
  4.  DT             2.65 ± 0.60
  5.  toasty         2.51 ± 0.97
  6.  may_hem        2.50 ± 0.41
  7.  Xx360NoSwagx   2.34 ± 0.66
  8.  Ritual         2.25 ± 0.67
  9.  phreak         2.22 ± 0.49
 10.  Shikari        2.18 ± 0.59
 11.  OuchMyBalls    2.17 ± 0.44
 12.  Alphachurro    2.15 ± 0.77
 13.  mex            2.11 ± 0.45
 14.  jig            2.08 ± 0.48
 15.  BALLDON'TLIE   2.06 ± 0.52
 16.  Suchit         2.04 ± 0.32
 17.  fender         2.02 ± 0.52
 18.  meowza         2.01 ± 0.47
 19.  realtea        1.93 ± 0.45
 20.  danp           1.91 ± 0.45
 21.  Messi          1.89 ± 0.42
 22.  Sif            1.81 ± 0.35
 23.  Enervate       1.81 ± 0.46
 24.  ft             1.80 ± 0.57
 25.  bright         1.79 ± 0.81
 26.  Ty             1.75 ± 0.77
 27.  rina           1.69 ± 0.58
 28.  tits           1.66 ± 0.76
 29.  1deag          1.63 ± 0.71
 30.  ASAP           1.63 ± 0.81
 31.  Dori

In [8]:
import random

ranks = []
for _ in range(10000):
    simulated_elos = {p: elo[p] + variance[p] ** 0.5 * random.gauss(0, 1) for p in elo if variance[p] <= 0.3}
    sorted_players = sorted(simulated_elos.items(), key=lambda x: x[1], reverse=True)
    ranks.append({player: rank + 1 for rank, (player, _) in enumerate(sorted_players)})

for i, player in enumerate(leaderboard[:]):
    rank_ci = sorted([r[player] for r in ranks])[500:-500]
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}  ({min(rank_ci)}-{max(rank_ci)})")

  1.  okthen         2.73 ± 0.67  (1-13)
  2.  SluffAndRuff   2.71 ± 0.62  (1-13)
  3.  CarrotCake     2.70 ± 0.30  (1-7)
  4.  DT             2.65 ± 0.60  (1-14)
  5.  toasty         2.51 ± 0.97  (1-34)
  6.  may_hem        2.50 ± 0.41  (2-14)
  7.  Xx360NoSwagx   2.34 ± 0.66  (2-29)
  8.  Ritual         2.25 ± 0.67  (3-34)
  9.  phreak         2.22 ± 0.49  (4-29)
 10.  Shikari        2.18 ± 0.59  (4-35)
 11.  OuchMyBalls    2.17 ± 0.44  (5-29)
 12.  Alphachurro    2.15 ± 0.77  (3-46)
 13.  mex            2.11 ± 0.45  (6-33)
 14.  jig            2.08 ± 0.48  (6-36)
 15.  BALLDON'TLIE   2.06 ± 0.52  (6-39)
 16.  Suchit         2.04 ± 0.32  (9-31)
 17.  fender         2.02 ± 0.52  (7-42)
 18.  meowza         2.01 ± 0.47  (8-40)
 19.  realtea        1.93 ± 0.45  (10-43)
 20.  danp           1.91 ± 0.45  (10-45)
 21.  Messi          1.89 ± 0.42  (11-45)
 22.  Sif            1.81 ± 0.35  (15-47)
 23.  Enervate       1.81 ± 0.46  (13-53)
 24.  ft             1.80 ± 0.57  (10-62)
 25.  brigh